In [1]:
library(quantmod)
library(fGarch)
library(repr)
library(forecast)
library(aTSA)
library(tseries)
library(MLmetrics)

Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 
Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.
Loading required package: timeDate
Loading required package: timeSeries

Attaching package: ‘timeSeries’

The following object is masked from ‘package:zoo’:

    time<-

Loading required package: fBasics

Attaching package: ‘fBasics’

The following object is masked from ‘package:TTR’:

    volatility

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 methods overwritten by 'forecast':
  method             from    
  fitted.fracdiff    fracdiff
  residual

In [2]:
install.packages("rugarch", repo = "https://cran.r-project.org/web/packages/rugarch/index.html")

also installing the dependencies ‘kernlab’, ‘mclust’, ‘multicool’, ‘mvtnorm’, ‘DistributionUtils’, ‘GeneralizedHyperbolic’, ‘Rsolnp’, ‘nloptr’, ‘ks’, ‘spd’, ‘chron’, ‘SkewHyperbolic’, ‘expm’

Warning message in install.packages("rugarch"):
“installation of package ‘mclust’ had non-zero exit status”Warning message in install.packages("rugarch"):
“installation of package ‘nloptr’ had non-zero exit status”Warning message in install.packages("rugarch"):
“installation of package ‘ks’ had non-zero exit status”Warning message in install.packages("rugarch"):
“installation of package ‘rugarch’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [ ]:
library(rugarch)

In [ ]:
getSymbols('BABA', src = 'yahoo', return.class = 'xts',from = "2014-09-20",to="2019-12-31")
head(BABA)
BABA <- BABA[,"BABA.Close"]
plot.xts(BABA, ylab = NA)

clearly non-stationary, lets calculate log returns

In [ ]:
BABA_logret <- na.omit(diff(log(BABA)))

plot.xts(BABA_logret, ylab = NA)
hist(BABA_logret,freq=FALSE,breaks=100)
curve(dnorm(x, mean=mean(BABA_logret), sd=sd(BABA_logret)), add=TRUE, col="red")

In [ ]:
adf <- data.frame("lags"=1:10,"p-val"=NA)

for (i in 1:10){
    adf[i,"p.val"] =as.numeric(adf.test(BABA_logret, k = i)$p.val)  
}
adf

stationary, lets examine ACF and PACF

In [ ]:
par(mfrow = c(2, 1))
acf(BABA_logret)
pacf(BABA_logret)

Also lets test joint significance of lags

In [ ]:
LB <- data.frame("lags"=1:10,"p-val"=NA)

for (i in 1:10){
    LB[i,"p.val"] =Box.test(BABA_logret, type = "Ljung-Box", lag = i)$p.val
}
LB

Null hypothesis of autocorrelations up to lag k equal zero is very likely to be rejected (according to Ljung box test)

From ACF and PACF as well as Box-Ljung test it appears that there is no dependence on lags in both subsamples and therefore I will proceed to GARCH.

Now, lets fit only mean and test for heteroskedasticity

In [ ]:
arima101 <- arima(BABA_logret, order = c(1, 0, 1))
arima101
arch.test(arima101)

In [ ]:
custom_ARIMA <- function(dat,max_p,max_q, d, const){
  model_config <-c()
  model_AIC <-c()
  model_BIC <-c()
  Ljung_Box_pval <-c()
  RMSE <- c()
  for (i in 1:(max_p+1)){
    for (j in 1:(max_q+1)){
          arima<- tryCatch(Arima(dat,  # variable
                                 order = c(i-1,d,j-1),  # (p,d,q) parameters
                                 include.constant = const),
                           warning = function(w) {print(paste("non-finite finite-difference value", ''));
                             NaN},
                           error = function(e) {print(paste("non-finite finite-difference value", ''));
                             NaN})
          
          print(paste(i-1,d,j-1,sep = '_'))
          if (!is.numeric(arima)){
            model_config <-c(model_config, 
                             paste(i-1,d,j-1,sep=',')
                             )
            model_AIC <- c(model_AIC, (AIC(arima)))
            model_BIC <- c(model_BIC, (BIC(arima)))
            Ljung_Box_pval <-c(Ljung_Box_pval,(as.numeric(sub(".*p-value = ","", 
                                                              capture.output(checkresiduals(arima, plot=FALSE)[5]))[5])))
            RMSE_tmp <-round(sqrt(mean((as.vector(arima$fitted) - as.vector(dat))^2)),1)
            RMSE <- c(RMSE, RMSE_tmp)
          }
        }
      }
  df <- as.data.frame(cbind(model_config, 
                            model_AIC,
                            as.numeric(as.character(model_BIC)),
                            as.numeric(as.character(Ljung_Box_pval)),
                            RMSE))
  names(df) <- c('model_config','model_AIC','model_BIC','Ljung_Box_pval','RMSE')
  df$model_config <- as.character(df$model_config)
  df$model_AIC <- as.numeric(as.character(df$model_AIC))
  df$model_BIC <- as.numeric(as.character(df$model_BIC))
  df$Ljung_Box_pval <- as.numeric(as.character(df$Ljung_Box_pval))
  return(df)
}

custom_ARIMA_results <- custom_ARIMA(BABA_logret, 10, 10, 0, FALSE)


In [ ]:
head(custom_ARIMA_results[order(custom_ARIMA_results$model_AIC),],5)

head(custom_ARIMA_results[order(custom_ARIMA_results$model_BIC),],5)

In [ ]:
auto.arima(BABA_logret, ic = "aic")
auto.arima(BABA_logret, ic = "bic")

# Conditional volatility

We can see that mean is very close to 0 (we might have white noise processes), and since we reject homoskedasticity in both subsamples, it is appropriate to fit model from GARCH family.

We start with GARCH(1,1) and set last 200 observations to be kept for out of sample forecasting 

In [ ]:
garch11_spec = ugarchspec(mean.model = list(armaOrder=c(0, 0)), variance.model = list(model = "sGARCH", garchOrder = c(1, 1)))

garch11_s1 = ugarchfit(garch11_spec, INTC_s1_logret, out.sample = 200)
garch11_s2 = ugarchfit(garch11_spec, INTC_s2_logret, out.sample = 200)

garch11_s1
garch11_s2

From the results we can see that all coefficients except mean are significant on both subsamples and null hypothesis of no autocorellation cannot be rejected (Ljung Box test). 

From test for normality we get

In [ ]:
jarque.bera.test(residuals(garch11_s1))

par(mfrow = c(1, 2))

hist(residuals(garch11_s1), breaks = 30, main ='Histogram', cex.main = 0.8, cex.lab = 0.8, xlab = NA,
    cex.axis = 0.8)
box()

qqnorm(residuals(garch11_s1), cex.main = 0.8, cex.lab = 0.8, cex.axis = 0.8) 
qqline(residuals(garch11_s1), lwd = 2)

In [ ]:
jarque.bera.test(residuals(garch11_s2))

par(mfrow = c(1, 2))

hist(residuals(garch11_s2), breaks = 30, main ='Histogram', cex.main = 0.8, cex.lab = 0.8, xlab = NA,
    cex.axis = 0.8)
box()

qqnorm(residuals(garch11_s2), cex.main = 0.8, cex.lab = 0.8, cex.axis = 0.8) 
qqline(residuals(garch11_s2), lwd = 2)

residuals do not appear to be normally distributed, lets fit residuals having students t-distribution

In [ ]:
garch11t_spec = ugarchspec(mean.model = list(armaOrder=c(0, 0)), variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),distribution.model = 'std')

garch11t_s1 = ugarchfit(garch11t_spec, INTC_s1_logret, out.sample = 200)
garch11t_s2 = ugarchfit(garch11t_spec, INTC_s2_logret, out.sample = 200)

garch11t_s1
garch11t_s2

In [ ]:
jarque.bera.test(residuals(garch11t_s1))

par(mfrow = c(1, 2))

hist(residuals(garch11t_s1), breaks = 30, main ='Histogram', cex.main = 0.8, cex.lab = 0.8, xlab = NA,
    cex.axis = 0.8)
box()

qqnorm(residuals(garch11t_s1), cex.main = 0.8, cex.lab = 0.8, cex.axis = 0.8) 
qqline(residuals(garch11t_s1), lwd = 2)

In [ ]:
jarque.bera.test(residuals(garch11t_s2))

par(mfrow = c(1, 2))

hist(residuals(garch11t_s2), breaks = 30, main ='Histogram', cex.main = 0.8, cex.lab = 0.8, xlab = NA,
    cex.axis = 0.8)
box()

qqnorm(residuals(garch11t_s2), cex.main = 0.8, cex.lab = 0.8, cex.axis = 0.8) 
qqline(residuals(garch11t_s2), lwd = 2)

We can see that there is no significant improvement.

Lets try to fit GARCH that accounts for asymmetric responses, namely EGEARCH and GJRGARCH

In [ ]:
egearch_spec <- ugarchspec(mean.model = list(armaOrder=c(0, 0)), variance.model = list(model = "eGARCH", 
                      garchOrder = c(1, 1)))


egearch_s1 = ugarchfit(egearch_spec, INTC_s1_logret, out.sample = 200)
egearch_s2 = ugarchfit(egearch_spec, INTC_s2_logret, out.sample = 200)

egearch_s1
egearch_s2

and EGEARCH with t dstribution

In [ ]:
egearcht_spec <- ugarchspec(mean.model = list(armaOrder=c(0, 0)), variance.model = list(model = "eGARCH", 
                      garchOrder = c(1, 1)),distribution.model = 'std')


egearcht_s1 = ugarchfit(egearcht_spec, INTC_s1_logret, out.sample = 200)
egearcht_s2 = ugarchfit(egearcht_spec, INTC_s2_logret, out.sample = 200)

egearcht_s1
egearcht_s2

and GJR_GARCH (both with normal and t distribution)

In [ ]:
gjrgarch_spec <- ugarchspec(mean.model = list(armaOrder=c(0, 0)), variance.model = list(model = "gjrGARCH", 
                      garchOrder = c(1, 1)))
gjrgarcht_spec <- ugarchspec(mean.model = list(armaOrder=c(0, 0)), variance.model = list(model = "gjrGARCH", 
                      garchOrder = c(1, 1)),distribution.model = 'std')

gjrgarch_s1 = ugarchfit(gjrgarch_spec, INTC_s1_logret, out.sample = 200)
gjrgarch_s2 = ugarchfit(gjrgarch_spec, INTC_s2_logret, out.sample = 200)
gjrgarcht_s1 = ugarchfit(gjrgarcht_spec, INTC_s1_logret, out.sample = 200)
gjrgarcht_s2 = ugarchfit(gjrgarcht_spec, INTC_s2_logret, out.sample = 200)

gjrgarch_s1
gjrgarch_s2
gjrgarcht_s1
gjrgarcht_s2

Lets compare coefficients from all models

In [ ]:
coefficients_s1 <- cbind(coef(garch11_s1),
                        coef(garch11t_s1),
                        coef(egearch_s1),
                        coef(egearcht_s1),  
                        coef(gjrgarch_s1),
                        coef(gjrgarcht_s1))

coefficients_s2 <- cbind(coef(garch11_s2),
                        coef(garch11t_s2),
                        coef(egearch_s2),
                        coef(egearcht_s2),  
                        coef(gjrgarch_s2),
                        coef(gjrgarcht_s2))

colnames(coefficients_s1) <- c("GARCH(1,1)","GARCH(1,1)-t","EGEARCH","EGEARCH-t","GJRGARCH","GJRGARCH-t")
colnames(coefficients_s2) <- c("GARCH(1,1)","GARCH(1,1)-t","EGEARCH","EGEARCH-t","GJRGARCH","GJRGARCH-t")
coefficients_s1
coefficients_s2

And lets check which model performs the best according to AIC

In [ ]:
performance <- cbind(c(infocriteria(garch11_s1)["Akaike",],
                        infocriteria(garch11t_s1)["Akaike",],
                        infocriteria(egearch_s1)["Akaike",],
                        infocriteria(egearcht_s1)["Akaike",],
                        infocriteria(gjrgarch_s1)["Akaike",],
                        infocriteria(gjrgarcht_s1)["Akaike",]),
                     c(infocriteria(garch11_s2)["Akaike",],
                        infocriteria(garch11t_s2)["Akaike",],
                        infocriteria(egearch_s2)["Akaike",],
                        infocriteria(egearcht_s2)["Akaike",],
                        infocriteria(gjrgarch_s2)["Akaike",],
                        infocriteria(gjrgarcht_s2)["Akaike",]))
colnames(performance) <- c("subsample 1","subsample 2")
rownames(performance) <- c("GARCH(1,1)","GARCH(1,1)-t","EGEARCH","EGEARCH-t","GJRGARCH","GJRGARCH-t")
performance


GARCH(1,1)-t and EGEARCH-t perform the best in both subsamples (lowest AIC), but since I expect the first one to be more appropriate, I will proceed with GARCH(1,1)-t

Now lets perform dynamic forecast and set the forecast horizon to 200

In [ ]:
for_garch11t_s1 = ugarchforecast(garch11t_s1, n.ahead = 1, n.roll = 200)
for_garch11t_s2 = ugarchforecast(garch11t_s2, n.ahead = 1, n.roll = 200)

In [ ]:
plot.ts(garch11t_s1@fit$sigma, ylab = NA, xlim = c(0, length(INTC_s1_logret)), main = 'GARCH(1,1) volatility forecasted for first subsample ')
lines(c(rep(NA, length(INTC_s1_logret) - 200 - 1), for_garch11t_s1@forecast$sigma), col = 'red')


plot.ts(garch11t_s2@fit$sigma, ylab = NA, xlim = c(0, length(INTC_s2_logret)), main = 'GARCH(1,1) volatility forecasted for second subsample')
lines(c(rep(NA, length(INTC_s2_logret) - 200 - 1), for_garch11t_s2@forecast$sigma), col = 'red')


Now, lets fit EWMA model (this is done by restricting omega to be equal to 0)

In [ ]:
ewma_spec = ugarchspec(mean.model = list(armaOrder=c(0, 0)), variance.model = list(model = "iGARCH", garchOrder = c(1, 1)),
                       fixed.pars = list(omega = 0))

ewma_s1 = ugarchfit(ewma_spec,INTC_s1_logret , out.sample = 200)
ewma_s2 = ugarchfit(ewma_spec,INTC_s2_logret, out.sample = 200)

and forecast

In [ ]:
for_ewma_s1 = ugarchforecast(ewma_s1, n.ahead = 1, n.roll = 200)
for_ewma_s2 = ugarchforecast(ewma_s2, n.ahead = 1, n.roll = 200)

In [ ]:
plot.ts(ewma_s1@fit$sigma, ylab = NA, xlim = c(0, length(INTC_s1_logret)), main = 'EWMA volatility forecasted for first subsample ')
lines(c(rep(NA, length(INTC_s1_logret) - 200 - 1), for_ewma_s1@forecast$sigma), col = 'red')


plot.ts(ewma_s2@fit$sigma, ylab = NA, xlim = c(0, length(INTC_s2_logret)), main = 'EWMA volatility forecasted for second subsample')
lines(c(rep(NA, length(INTC_s2_logret) - 200 - 1), for_ewma_s2@forecast$sigma), col = 'red')


From the graphs we can see that they produce similar predictions (especially in second subsample), lets examine MSE.

First I bind the prediction vectors

In [ ]:
predictions  <- cbind(for_garch11t_s1@forecast$sigmaFor[1:200],
                      for_garch11t_s2@forecast$sigmaFor[1:200],
                      for_ewma_s1@forecast$sigmaFor[1:200],
                      for_ewma_s1@forecast$sigmaFor[1:200])

Then I calculate volatility proxy

In [ ]:
vol_s1 <- (tail(INTC_s1_logret,200))^2
vol_s2 <- (tail(INTC_s2_logret,200))^2

And finally proceed to MSE

In [ ]:
MSE_results <- cbind(MSE(y_pred = predictions[1], y_true = vol_s1),
                     MSE(y_pred = predictions[2], y_true = vol_s2),
                     MSE(y_pred = predictions[3], y_true = vol_s1),
                     MSE(y_pred = predictions[4], y_true = vol_s2))


colnames(MSE_results) <- c('GARCH(1,1)-t subsample 1',
                          'GARCH(1,1)-t subsample 2',
                          'EWMA subsample 1',
                          'EWMA subsample 2')

MSE_results

GARCH(1,1)-t appears to perform better than EWMA in terms of mean squared error in both subsamples